In [1]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

model = Wav2Vec2ForCTC.from_pretrained("./wav2vec2-large-xlsr-ger-chris-cut/checkpoint-51000")

In [2]:
def map_to_result(batch):
    
    model.to("cuda")
    input_values = processor(
          batch["speech"], 
          sampling_rate=batch["sampling_rate"], 
          return_tensors="pt"
    ).input_values.to("cuda")

    with torch.no_grad():
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]

    return batch

In [3]:
from datasets import load_metric

wer_metric = load_metric("wer")

In [4]:
from datasets import load_from_disk, load_dataset

test_sampled = load_from_disk("E:/Master/data/test_sampled")

In [5]:
test_sampled.shape

(15588, 3)

In [5]:
results51_cut = test_sampled.map(map_to_result)

KeyboardInterrupt: 

In [7]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["target_text"])))

Test WER: 0.140


In [7]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results51_cut["pred_str"], references=results51_cut["target_text"])))

Test WER: 0.148


In [8]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [12]:
results51_cut.save_to_disk("results51_cut")

In [6]:
results51_cut = load_from_disk("results51_cut")

In [9]:
show_random_elements(results.remove_columns(["speech", "sampling_rate"]))

NameError: name 'results' is not defined

In [10]:
show_random_elements(results51_cut.remove_columns(["speech", "sampling_rate"]))

,pred_str,target_text
0,sein grab befindet sich auf dem friedhof wilmersdorf in berlin,sein grab befindet sich auf dem friedhof wilmersdorf in berlin
1,in ihm wurde salz gelagert,in ihnen wurde salz gelagert
2,eine obligatorische kopular gibt es nicht,eine obligatorische kopula gibt es nicht
3,hält dieser zug auch am hauptbahnhof,hält dieser zug auch am hauptbahnhof
4,kurz darauf erschieß sich detley,kurz darauf erschießt sich tetley
5,der kraftstofffilter ist vollkommen verstopft,der kraftstofffilter ist vollkommen verstopft
6,simus folgeleistenst,sie muss folge leisten
7,wilbin sldie sorten dare trimrich weiden,irgendwelche deppen warfen chinaböller in die menge
8,auch bei dtruskern war die darstellung des archilus in der kunst weit verbreitet,auch bei den etruskern war die darstellung des acheloos in der kunst weit verbreitet
9,sie gilt als die älteste glockengieserei in england,sie gilt als die älteste glockengießerei in england


In [11]:
!pip show transformers

Name: transformers
Version: 4.7.0.dev0
Summary: State-of-the-art Natural Language Processing for TensorFlow 2.0 and PyTorch
Home-page: https://github.com/huggingface/transformers
Author: Thomas Wolf, Lysandre Debut, Victor Sanh, Julien Chaumond, Sam Shleifer, Patrick von Platen, Sylvain Gugger, Suraj Patil, Stas Bekman, Google AI Language Team Authors, Open AI team Authors, Facebook AI Authors, Carnegie Mellon University Authors
Author-email: thomas@huggingface.co
License: Apache
Location: d:\transformers\src
Requires: filelock, huggingface-hub, numpy, packaging, regex, requests, sacremoses, tokenizers, tqdm
Required-by: spacy-transformers
